<a href="https://colab.research.google.com/github/lucaspn017/time_series_analysis/blob/main/Time_Series_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Separando os valores de consumo na ponta e fora-ponta </h1>

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

data = pd.read_excel('power.xlsx')
datafp = pd.read_excel('powerfactor.xlsx')

data['DateTime']= pd.to_datetime(data['DateTime'])
datafp['DateTime']= pd.to_datetime(datafp['DateTime'])

data.set_axis(data['DateTime'], inplace=True)

point = data[(data['DateTime'].dt.time >= datetime.strptime("18:30:00", '%H:%M:%S').time()) & (data['DateTime'].dt.time <= datetime.strptime("21:29:00", '%H:%M:%S').time())]
outPoint = data[(data['DateTime'].dt.time >= datetime.strptime("22:30:00", '%H:%M:%S').time()) & (data['DateTime'].dt.time <= datetime.strptime("23:50:00", '%H:%M:%S').time()) | (data['DateTime'].dt.time >= datetime.strptime("00:00:00", '%H:%M:%S').time()) & (data['DateTime'].dt.time <= datetime.strptime("17:29:00", '%H:%M:%S').time())]
inter = (data.drop(point.index)).drop(outPoint.index)


pointfp = datafp[(datafp['DateTime'].dt.time >= datetime.strptime("18:30:00", '%H:%M:%S').time()) & (datafp['DateTime'].dt.time <= datetime.strptime("21:29:00", '%H:%M:%S').time())]
outPointfp = datafp[(datafp['DateTime'].dt.time >= datetime.strptime("22:30:00", '%H:%M:%S').time()) & (datafp['DateTime'].dt.time <= datetime.strptime("23:50:00", '%H:%M:%S').time()) | (datafp['DateTime'].dt.time >= datetime.strptime("00:00:00", '%H:%M:%S').time()) & (datafp['DateTime'].dt.time <= datetime.strptime("17:29:00", '%H:%M:%S').time())]
interfp = (datafp.drop(pointfp.index)).drop(outPointfp.index)

#OBTENÇÃO DOS VALORES DE PONTA

value = point['Total'].tolist()
value = np.array(value)
consP = np.trapz(value, dx=1/6)

#OBTENÇÃO DOS VALORES FORA-PONTA

value = outPoint['Total'].tolist()
value = np.array(value)
consFP = np.trapz(value, dx=1/6)

print(f'O VALOR DE CONSUMO NO HORÁRIO DE PONTA É: {consP} Wh \n\nO VALOR DE CONSUMO NO HORÁRIO FORA-PONTA É: {consFP} Wh')


<h1> Separando os valores de consumo de reativo na ponta e fora-ponta </h1>

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

data = pd.read_excel('power.xlsx')
datafp = pd.read_excel('powerfactor.xlsx')

data['DateTime']= pd.to_datetime(data['DateTime'])
datafp['DateTime']= pd.to_datetime(datafp['DateTime'])

data.set_axis(data['DateTime'], inplace=True)
datafp.set_axis(data['DateTime'], inplace=True)

# Separando os valores de potência ativa em Ponta, Fora-Ponta e Intermediário
point = data[(data['DateTime'].dt.time >= datetime.strptime("18:30:00", '%H:%M:%S').time()) & (data['DateTime'].dt.time <= datetime.strptime("21:29:00", '%H:%M:%S').time())]
outPoint = data[(data['DateTime'].dt.time >= datetime.strptime("22:30:00", '%H:%M:%S').time()) & (data['DateTime'].dt.time <= datetime.strptime("23:50:00", '%H:%M:%S').time()) | (data['DateTime'].dt.time >= datetime.strptime("00:00:00", '%H:%M:%S').time()) & (data['DateTime'].dt.time <= datetime.strptime("17:29:00", '%H:%M:%S').time())]
inter = (data.drop(point.index)).drop(outPoint.index)

# Separando os valores de fator de potência em Ponta, Fora-Ponta e Intermediário
pointfp = datafp[(datafp['DateTime'].dt.time >= datetime.strptime("18:30:00", '%H:%M:%S').time()) & (datafp['DateTime'].dt.time <= datetime.strptime("21:29:00", '%H:%M:%S').time())]
outPointfp = datafp[(datafp['DateTime'].dt.time >= datetime.strptime("22:30:00", '%H:%M:%S').time()) & (datafp['DateTime'].dt.time <= datetime.strptime("23:50:00", '%H:%M:%S').time()) | (datafp['DateTime'].dt.time >= datetime.strptime("00:00:00", '%H:%M:%S').time()) & (datafp['DateTime'].dt.time <= datetime.strptime("17:29:00", '%H:%M:%S').time())]
interfp = (datafp.drop(pointfp.index)).drop(outPointfp.index)

def calc_reat_exc(**kwargs):
    fr=0.92
    reat_exc = 0
    reat_exc2 = 0

    tempoI1 = datetime.strptime("23:30:00","%H:%M:%S").time()
    tempoF1 = datetime.strptime("23:50:00","%H:%M:%S").time()
    tempoI2 = datetime.strptime("00:00:00","%H:%M:%S").time()
    tempoF2 = datetime.strptime("06:30:00","%H:%M:%S").time()

    fpD = kwargs['fp_df']
    apD = kwargs['ap_df']

    #Para valores na ponta
    for index in fpD.index:
        if 0 <fpD[index] < 0.92:
            reat_exc = reat_exc + apD[index]*(fr/abs(fpD[index]) - 1)

    #Para valores fora-ponta(Pois é apenas entre às 11:30 e 06:30:00)

    filt = kwargs['fp_df'][(kwargs['fp_df'].index.time >= tempoI1) & (kwargs['fp_df'].index.time <= tempoF1)] & kwargs['fp_df'][(kwargs['fp_df'].index.time >= tempoI2) & (kwargs['fp_df'].index.time <= tempoF2)]
    fiapD = kwargs['ap_df'][(kwargs['ap_df'].index.time >= tempoI1) & (kwargs['ap_df'].index.time <= tempoF1)] & kwargs['ap_df'][(kwargs['ap_df'].index.time >= tempoI2) & (kwargs['ap_df'].index.time <= tempoF2)]
    fpD = kwargs['fp_df'].drop(filt.index)
    apD = kwargs['ap_df'].drop(fiapD.index)

    for index in filt.index:
        if 0 > filt[index] > -0.92:
            reat_exc2 = reat_exc2 + filt[index]*(fr/abs(filt[index]) - 1)
    total = reat_exc + reat_exc2

    return total

point = point.resample('1H').mean()
pointfp = pointfp.resample('1H').mean()

outPoint = outPoint.resample('1H').mean()
outPointfp = outPointfp.resample('1H').mean()

dataRU = data.resample('1H').mean()

calc_reat_exc(fp_df =  pointfp['Total'], ap_df = point['Total'])

#Forma alternativa de obter um intevalo de dados
#Não é possível comparar o valor obtido com o strptime, se não for utilizado o atributo .time no DataFrame point com atributo .index.
